# Brain Tumor Detection using Mask R-CNN 

# 1-Import Libraries

In [ ]:
pip install -U scikit-image==0.16.2

In [ ]:
import os 
import sys
from tqdm import tqdm
import cv2
import numpy as np
import json
import skimage.draw
import matplotlib
import matplotlib.pyplot as plt
import random
from pathlib import Path
from IPython.display import clear_output

DATASET_DIR = '/kaggle/input/palm-oil-json/palm-oil'
ROOT_DIR = Path('/kaggle/working')
#-------- directory to save logs and trained model----
MODEL_DIR = os.path.join(ROOT_DIR, 'logs')
DEFAULT_LOGS_DIR = 'logs' 

# 2-Installing

## 2.1-Install the previous version of TensorFlow(1.14)

In [ ]:
!pip install tensorflow==1.14
import tensorflow as tf
clear_output()

In [ ]:
tf.__version__

## 2.2 Install Keras version (2.1.5)

In [ ]:
!pip install keras==2.1.5
clear_output()

## 2.3 Install pycocotools

In [ ]:
!pip install pycocotools
clear_output()

# 3-Download the Model Mask_RCNN 

In [ ]:
!git clone https://www.github.com/matterport/Mask_RCNN.git
!rm -rf .git # to prevent an error when the kernel is committed
!rm -rf images assets # to prevent displaying images at the bottom of a kernel
clear_output()

In [ ]:
# Root directory of the project
ROOT_DIR = os.path.abspath('Mask_RCNN/')
# Import Mask RCNN
sys.path.append(ROOT_DIR) 
from mrcnn.config import Config
from mrcnn import utils
from mrcnn.model import log
import mrcnn.model as modellib
from mrcnn import visualize

clear_output()

# 4-Import COCO config

In [ ]:
sys.path.append(os.path.join(ROOT_DIR, 'samples/coco/'))
import coco

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)
plt.rcParams['figure.facecolor'] = 'white'

clear_output()

In [ ]:
class TumorConfig(Config):
    NAME = 'Brain_Tumor_Detector'
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 1 
    DETECTION_MIN_CONFIDENCE = 0.9    
    STEPS_PER_EPOCH = 80
    LEARNING_RATE = 0.005    
config = TumorConfig()
config.display()

# 5-Brain Tumor Dataset

In [ ]:
class BrainTumorDataset(utils.Dataset):
    def load_brain_scan(self, dataset_dir, subset):
        """Load a subset of the FarmCow dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("tumor", 1, "tumor")

        # Train or validation dataset?
        assert subset in ["train", "valid", 'test']
        dataset_dir = os.path.join(dataset_dir, subset)

        annotations = json.load(open(os.path.join(dataset_dir,'annotation_json'+'.json')))
        annotations = list(annotations.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]

        # Add images
        for a in annotations:
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. These are stores in the
            # shape_attributes (see json format above)
            # The if condition is needed to support VIA versions 1.x and 2.x.
            if type(a['regions']) is dict:
                polygons = [r['shape_attributes'] for r in a['regions'].values()]
            else:
                polygons = [r['shape_attributes'] for r in a['regions']]

            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            image_path = os.path.join(dataset_dir, a['filename'])
            
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "tumor",
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, 
                height=height,
                polygons=polygons
            )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a farm_cow dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "tumor":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "tumor":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
# Training dataset.
dataset_train = BrainTumorDataset()
dataset_train.load_brain_scan(DATASET_DIR, 'train')
dataset_train.prepare()

# Validation dataset
dataset_val = BrainTumorDataset()
dataset_val.load_brain_scan(DATASET_DIR, 'valid')
dataset_val.prepare()

dataset_test = BrainTumorDataset()
dataset_test.load_brain_scan(DATASET_DIR, 'test')
dataset_test.prepare()

# 6-Visualizing Some Samples

In [ ]:
print(dataset_train.image_reference(6))
image = dataset_train.load_image(6)
mask, class_ids = dataset_train.load_mask(6)
visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names, limit=1)
    

In [ ]:
print(dataset_train.image_reference(30))
image = dataset_train.load_image(30)
mask, class_ids = dataset_train.load_mask(30)
visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names, limit=1)

In [ ]:
print(dataset_train.image_reference(40))
image = dataset_train.load_image(40)
mask, class_ids = dataset_train.load_mask(40)
visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names, limit=1)

# 7-Build the Mask R-CNN Model Architecture


In [ ]:
#-------------- Create model object in training mode------------------------
model = modellib.MaskRCNN(
    mode='training', 
    config=config, 
    model_dir=DEFAULT_LOGS_DIR
)
#-------------- Load weights trained on MS-COCO ---------------------------
model.load_weights(
    COCO_MODEL_PATH, 
    by_name=True, 
    exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"]
)

# 8-Training

In [ ]:
model.train(
    dataset_train, dataset_val,
    learning_rate=config.LEARNING_RATE,
    epochs=10,
    layers='heads'
)

# 9-Helper Functions


In [ ]:
def display_image(dataset, ind):
    plt.figure(figsize=(5,5))
    plt.imshow(dataset.load_image(ind))
    plt.xticks([])
    plt.yticks([])
    plt.title('Original Image')
    plt.show()
def get_ax(rows=1, cols=1, size=5):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

def predict_and_plot_differences(dataset, img_id):
    original_image, image_meta, gt_class_id, gt_box, gt_mask =\
        modellib.load_image_gt(dataset, config, 
                               img_id, use_mini_mask=False)

    results = model.detect([original_image], verbose=0)
    r = results[0]

    visualize.display_differences(
        original_image,
        gt_box, gt_class_id, gt_mask,
        r['rois'], r['class_ids'], r['scores'], r['masks'],
        class_names = ['tumor'], title="", ax=get_ax(),
        show_mask=True, show_box=True)

def calculate_areas(masks):
    """Calculate the area of each mask."""
    areas = []
    for i in range(masks.shape[-1]):
        mask = masks[:, :, i]
        area = np.sum(mask)
        areas.append(area)
    return areas

def predict(dataset, img_id):
    # Load the image and run detection
    original_image, image_meta, gt_class_id, gt_box, gt_mask =\
        modellib.load_image_gt(dataset, config, 
                               img_id, use_mini_mask=False)

    results = model.detect([original_image], verbose=0)
    r = results[0]

    # Calculate areas of detected items
    areas = calculate_areas(r['masks'])

    # Display predictions
    visualize.display_instances(
        original_image,
        r['rois'], r['masks'], r['class_ids'], 
        dataset.class_names, r['scores'],
        title="Predicted Results", ax=get_ax(),
        show_mask=True, show_bbox=True)

    # Print the area of each detected item
    for i, area in enumerate(areas):
        print(f"Item {i + 1}: Area = {area} pixels")

# Assuming modellib and visualize are already imported and configured

# 10-Inference

In [ ]:
# Recreate the model in inference mode
model = modellib.MaskRCNN(
    mode="inference", 
    config=config,
    model_dir=DEFAULT_LOGS_DIR
)


model_path = model.find_last()
#model_path="../input/mask-r-cnn-tumor-brain/logs/tumor_detector20210209T1932/mask_rcnn_tumor_detector_0002.h5"
# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

# 11-Detect Tumors and Visualize the Results

In [ ]:
print("Sample 1")
ind = 10
display_image(dataset_val, ind)
predict(dataset_val, ind)


In [ ]:
print("Sample 2")
ind = 16
display_image(dataset_val, ind)
predict_and_plot_differences(dataset_val, ind)

In [ ]:
print("Sample 3")
ind = 0
display_image(dataset_test, ind)
predict_and_plot_differences(dataset_test, ind)

In [ ]:
print("Sample 4")
ind = 2
display_image(dataset_test, ind)
predict_and_plot_differences(dataset_test, ind)

In [ ]:
print("Sample 5")
ind = 3
display_image(dataset_test, ind)
predict_and_plot_differences(dataset_test, ind)

# 12- References
1. https://www.kaggle.com/ruslankl/brain-tumor-detection-v2-0-mask-r-cnn
